# Tutorial: Learn the basics of the Druid API

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
This tutorial introduces you to the basics of the Druid API and some of the endpoints you might use frequently, including the following tasks:

- Checking if your cluster is up
- Ingesting data
- Querying data
- Deleting data

In a Druid deployment, you have [Mastery, Query, and Data servers](https://druid.apache.org/docs/latest/design/processes.html#server-types) that all fulfill different purposes. The endpoint you use for a certain action is determined, partially, by which server governs that part of Druid and the processes that run on that server type. That's why the [API reference](https://druid.apache.org/docs/latest/operations/api-reference.html#historical) is organized by server type and process.

## Table of contents

- [Before you start](#Before-you-start)
- [Get basic cluster information](#Get-basic-cluster-information)
- [Ingest data](#Ingest-data)
- [Query your data](#Query-your-data)
- [Manage your data](#Manage-your-data)
- [Next steps](#Next-steps)

For the best experience, use Jupyter Lab so that you can always access the table of contents.

## Requirements

You'll need install the Requests library for Python before you start. For example:

```bash
pip3 install requests
```

Next, you'll need a Druid cluster. This tutorial uses the `micro-quickstart` config described in the [Druid quickstart](https://druid.apache.org/docs/latest/tutorials/index.html). So download that and start it if you haven't already. In the root of the Druid folder, run the following command to start Druid:

```bash
./bin/start-micro-quickstart
```

Finally, you'll need either Jupyter lab (recommended) or Jupyter notebook. Both the quickstart Druid cluster and Jupyter notebook are deployed at `localhost:8888` by default, so you'll 
need to change the port for Jupyter. To do so, stop Jupyter and start it again with the `port` parameter included. For example, you can use the following command to start Jupyter on port `3001`:

```bash
# If you're using Jupyter lab
jupyter lab --port 3001
# If you're using Jupyter notebook
jupyter notebook --port 3001 
```

To start this tutorial, run the next cell. It imports the Python packages you'll need and defines a variable for the the Druid host the tutorial uses. The quickstart deployment configures Druid to listen on port `8888` by default, so you'll be making API calls against `http://localhost:8888`. 

In [ ]:
import requests
import json

# druid_host is the hostname and port for your Druid deployment. 
druid_host = "http://localhost:8888"
dataSourceName = "wikipedia_api"
print(druid_host)

In the rest of this tutorial, the `endpoint`, `http_method`, and `payload` variables are updated in code cells to call a different Druid endpoint to accomplish a task.

## Get basic cluster information

In this cell, you'll use the `GET /status` endpoint to return basic information about your cluster, such as the Druid version, loaded extensions, and resource consumption.

The following cell sets `endpoint` to `/status` and updates the HTTP method to `GET`. When you run the cell, you should get a response that starts with the version number of your Druid deployment.

In [ ]:
endpoint = "/status"
print(druid_host+endpoint)
http_method = "GET"

payload = {}
headers = {}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### Get cluster health

The `/status/health` endpoint returns `true` if your cluster is up and running. It's useful if you want to do something like programmatically check if your cluster is available. When you run the following cell, you should get `true` if your Druid cluster has finished starting up and is running.

In [ ]:
# GET 
endpoint = "/status/health"
print(druid_host+endpoint)
http_method = "GET"

payload = {}
headers = {}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)

print(response.text)

## Ingest data

Now that you've confirmed that your cluster is up and running, you can start ingesting data. There are different ways to ingest data based on what your needs are. For more information, see [Ingestion methods](https://druid.apache.org/docs/latest/ingestion/index.html#ingestion-methods).

This tutorial uses the multi-stage query (MSQ) task engine and its `sql/task` endpoint to perform SQL-based ingestion. The `/sql/task` endpoint accepts [SQL requests in the JSON-over-HTTP format](https://druid.apache.org/docs/latest/querying/sql-api.html#request-body) using the query, context, and parameters fields

To learn more about SQL-based ingestion, see [SQL-based ingestion](https://druid.apache.org/docs/latest/multi-stage-query/index.html). For information about the endpoint specifically, see [SQL-based ingestion and multi-stage query task API](https://druid.apache.org/docs/latest/multi-stage-query/api.html).


The next cell does the following:

- Includes a payload that inserts data from an external source into a table named wikipedia_api. The payload is in JSON format and included in the code directly. You can also store it in a file and provide the file. 
- Saves the response to a unique variable that you can reference later to identify this ingestion task

The example uses INSERT, but you could also use REPLACE. 

For the MSQ task engine, ingesting data is done through a task, so the response includes a `taskId` and `state` for your ingestion. You can use this `taskId` to reference this task later on to get more information about it.

In [ ]:
endpoint = "/druid/v2/sql/task"
print(druid_host+endpoint)
http_method = "POST"


payload = json.dumps({
"query": "INSERT INTO wikipedia_api SELECT   TIME_PARSE(\"timestamp\") AS __time,   * FROM TABLE(   EXTERN(     '{\"type\": \"http\", \"uris\": [\"https://druid.apache.org/data/wikipedia.json.gz\"]}',     '{\"type\": \"json\"}',     '[{\"name\": \"added\", \"type\": \"long\"}, {\"name\": \"channel\", \"type\": \"string\"}, {\"name\": \"cityName\", \"type\": \"string\"}, {\"name\": \"comment\", \"type\": \"string\"}, {\"name\": \"commentLength\", \"type\": \"long\"}, {\"name\": \"countryIsoCode\", \"type\": \"string\"}, {\"name\": \"countryName\", \"type\": \"string\"}, {\"name\": \"deleted\", \"type\": \"long\"}, {\"name\": \"delta\", \"type\": \"long\"}, {\"name\": \"deltaBucket\", \"type\": \"string\"}, {\"name\": \"diffUrl\", \"type\": \"string\"}, {\"name\": \"flags\", \"type\": \"string\"}, {\"name\": \"isAnonymous\", \"type\": \"string\"}, {\"name\": \"isMinor\", \"type\": \"string\"}, {\"name\": \"isNew\", \"type\": \"string\"}, {\"name\": \"isRobot\", \"type\": \"string\"}, {\"name\": \"isUnpatrolled\", \"type\": \"string\"}, {\"name\": \"metroCode\", \"type\": \"string\"}, {\"name\": \"namespace\", \"type\": \"string\"}, {\"name\": \"page\", \"type\": \"string\"}, {\"name\": \"regionIsoCode\", \"type\": \"string\"}, {\"name\": \"regionName\", \"type\": \"string\"}, {\"name\": \"timestamp\", \"type\": \"string\"}, {\"name\": \"user\", \"type\": \"string\"}]'   ) ) PARTITIONED BY DAY",
  "context": {
    "maxNumTasks": 3
  }
})

headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
ingestiion_taskId_response = response
print(response.text + f"\nInserting data into the table named {dataSourceName}.")

Extract the `taskId` value from the `taskId_response` variable so that you can reference it later:

In [ ]:
ingestion_taskId = json.loads(ingestiion_taskId_response.text)['taskId']
print(ingestion_taskId)

### Get the status of your task

The following cell shows you how to get the status of your ingestion.  You can see basic information about your query, such as when it started and whether or not it's finished.

In addition to the status, you can retrieve a full report about it if you want using `GET /druid/indexer/v1/task/TASK_ID/reports`. But you won't need that information for this tutorial.

In [ ]:
endpoint = f"/druid/indexer/v1/task/{ingestion_taskId}/status"
print(druid_host+endpoint)
http_method = "GET"

payload = {}
headers = {}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Query your data

When you ingest data into Druid, Druid stores the data in a datasource, and this datasource is what you run queries against.

### List your datasources

You can get a list of datasources from the `/druid/coordinator/v1/datasources` endpoint. Since you're just getting started, there should only be a single datasource, the `wikipedia_api` table you created earlier when you ingested external data.

In [ ]:
endpoint = "/druid/coordinator/v1/datasources"
print(druid_host+endpoint)
http_method = "GET"

payload = {}
headers = {}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(response.text)

### Query your data

Now, you can query the data. Because this tutorial is running in Jupyter, make sure to limit the size of your query results using `LIMIT`. For example, the following cell selects all columns but limits the results to 3 rows for display purposes.


In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query": "SELECT  * FROM wikipedia_api LIMIT 3"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)

print(json.dumps(json.loads(response.text), indent=4))



In addition to the query, there are a few additional things you can define within the payload. For a full list, see [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html)

This tutorial uses a context parameter and a dynamic parameter.

Context parameters can control certain characteristics related to a query, such as configuring a custom timeout. For information, see [Context parameters](https://druid.apache.org/docs/latest/querying/query-context.html). In the example query that follows, the context block assigns a custom `sqlQueryID` to the query. Typically, the `sqlQueryId` is autogenerated. With a custom ID, you can use it to reference the query more easily like when you need to cancel a query.


Druid supports dynamic parameters, so you can either define certain parameters within the query explicitly or insert a `?` as a placeholder and define it in a parameters block. In the following cell, the `?` gets bound to the timestmap value of `2016-06-27` at execution time. For more information, see [Dynamic parameters](https://druid.apache.org/docs/latest/querying/sql.html#dynamic-parameters).


The following cell selects rows where the `__time` column contains a value greater than the value defined dynamically in `parameters` and sets a custom `sqlQueryId`.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query": "SELECT * FROM wikipedia_api WHERE __time > ? LIMIT 1",
  "context": {
    "sqlQueryId" : "important-query" 
    },
  "parameters": [
    { "type": "TIMESTAMP", "value": "2016-06-27"}
  ]
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Next steps

This tutorial covers the some of the basics related to the Druid API. To learn more about the kinds of things you can do, see the API documentation:

- [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html)
- [API reference](https://druid.apache.org/docs/latest/operations/api-reference.html)

You can also try out the [druid-client](https://github.com/paul-rogers/druid-client), a Python library for Druid created by a Druid contributor.



